In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

In [2]:
1+1

2

In [15]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","""You are a smart json builder, you will take natural language and input and convert it into json , return json only NO PREAMBLE
         """),
        ("human","CONTENT: {question}")
    ]
)

In [16]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [17]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    api_key=os.getenv("GOOGLE_API_KEY")
)

In [18]:
llm

ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001F3C37711D0>, default_metadata=())

In [ ]:
llm.invoke("question": "his name is ronith and hes from india")

AIMessage(content='The King of Bahrain is **Hamad bin Isa Al Khalifa**.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--76321eba-351e-4638-a92d-f9da397d42bd-0', usage_metadata={'input_tokens': 8, 'output_tokens': 14, 'total_tokens': 22, 'input_token_details': {'cache_read': 0}})

In [19]:
from langchain_core.output_parsers import JsonOutputParser

In [20]:
chain = prompt | llm | JsonOutputParser()

In [21]:
chain.invoke({"question": "his name is ronith and hes from india"})

{'name': 'Ronith', 'country': 'India'}

In [22]:
template =  """
    You are a helpful assistant that converts natural language queries into a structured JSON object for a customer segmentation system.

    Your task is to convert the following user query into a JSON object with a 'combinator' field and a 'rules' array. Each rule object must have 'field', 'operator', and 'value'.

    Here are the available fields and their data types from the customer database:
    - totalSpend (Number)
    - visits (Number)
    - lastPurchaseDate (Date)

    Here are the valid operators for each data type:
    - Numbers: >, <, >=, <=, ==, !=
    - Dates: >, <, ==, != (to compare dates)

    The valid combinators are '$and' and '$or'.

    Example Input:
    "Find customers who have spent over 500 dollars and visited less than 3 times."

    Example Output:
    {{
        "combinator": "$and",
        "rules": [
            {{"field": "totalSpend", "operator": ">", "value": 500}},
            {{"field": "visits", "operator": "<", "value": 3}}
        ]
    }}

    Always use the exact field names, operators, and combinators provided.
    For dates, use a relative format like 'X months ago' or a specific format 'YYYY-MM-DD'.
    If no combinator is specified, use '$and' by default.
    Do not add any additional text or explanations to the response.
    
    User Query: {user_query}
    """

In [24]:
new_prompt = ChatPromptTemplate.from_template(template)

new_chain = new_prompt | llm | JsonOutputParser()

new_chain.invoke({"user_query":"Find customers who have spent over 500 dollars and visited less than 3 times."})

{'combinator': '$and',
 'rules': [{'field': 'totalSpend', 'operator': '>', 'value': 500},
  {'field': 'visits', 'operator': '<', 'value': 3}]}